### Step 1 — Imports & sample data
We will prepare two sources: a random string and a repetitive patterned string to illustrate LZ77
behavior

In [23]:
import random
random.seed(0)  # fix RNG for reproducible results

def random_string(n=2000, alphabet='abcdefghijklmnopqrstuvwxyz '):
    """Return a length-n string of pseudo-random chars from alphabet (deterministic due to seed)."""
    return ''.join(random.choice(alphabet) for _ in range(n))  # pick n random chars and join

def repetitive_string(n=2000):
    """Return a length-n string made by repeating the pattern 'abracadabra '."""
    pattern = 'abracadabra '  # base repeating pattern (length 12)
    s = (pattern * (n // len(pattern) + 1))[:n]  # repeat enough times, then slice to n
    return s

S_rand = random_string(2000)  # pseudo-random 2000-char string
S_rep = repetitive_string(2000)  # periodic 2000-char string

len(S_rand), len(S_rep)  # sanity check: both should be exactly 2000

(2000, 2000)

### Step 2 — LZ77 encoder (sliding-window longest match)
We search backward within a finite window to find the longest match with the lookahead buffer.
We output a triple (offset, length, next_symbol). An offset of 0 and length 0 means the symbol is
new and we output it directly via next_symbol.

In [24]:
def lz77_encode(s, window_size=128, lookahead_size=32):
    """Encode string s into LZ77 triples (offset, length, next_symbol)."""
    i = 0                     # current position in s
    out = []                  # output list of triples
    n = len(s)

    while i < n:
        best_len = 0          # length of best match found so far
        best_off = 0          # offset (distance back) of best match
        start_hist = max(0, i - window_size)  # left bound of search window

        # Scan history window [start_hist, i) for the longest match with lookahead
        for j in range(start_hist, i):
            k = 0
            # Extend the match as long as chars are equal and within limits
            while (k < lookahead_size and i + k < n and s[j + k] == s[i + k]):
                k += 1
            # Keep the best (longest) match
            if k > best_len:
                best_len = k
                best_off = i - j
            # Early exit if lookahead is fully matched
            if best_len == lookahead_size:
                break

        # Next literal symbol after the matched run ('' if at end)
        next_sym = s[i + best_len] if i + best_len < n else ''
        out.append((best_off, best_len, next_sym))  # emit triple
        i += best_len + 1       # advance past the match and the next symbol

    return out

# Example: encode a short string with a small window/lookahead
triples = lz77_encode('ababaabababa', window_size=8, lookahead_size=6)
triples[:6]  # first 6 triples (offset, length, next_symbol)

[(0, 0, 'a'), (0, 0, 'b'), (2, 3, 'a'), (5, 4, 'b'), (7, 1, '')]

### Step 3 — LZ77 decoder
Decoder reconstructs the original string by copying 'length' characters from 'offset' back from the
current end and then appending the next_symbol (if present).

In [25]:
def lz77_decode(triples):
    """Decode LZ77 triples (offset, length, next_symbol) back to a string."""
    out = []  # grows as a list of chars for efficient appends
    for off, ln, c in triples:
        if off == 0 and ln == 0:
            out.append(c)  # literal (no back-reference)
        else:
            start = len(out) - off          # where to start copying from
            for k in range(ln):             # copy 'ln' chars from the history
                out.append(out[start + k])
            if c:                            # then append the next literal (if any)
                out.append(c)
    return ''.join(out)  # join all chars into the final string


# Round-trip test (encode -> decode)
s = 'ababaabababa'
enc = lz77_encode(s, window_size=8, lookahead_size=6)  # reuse the encoder above
dec = lz77_decode(enc)
print('Round-trip OK?', dec == s)  # should be True if encoder/decoder agree

Round-trip OK? True


### Step 4 — Bit cost model & metrics
To estimate compressed size (without writing a full bitstream), assume fixed-size fields:
⌈log2(window_size+1)⌉ bits for offset, ⌈log2(lookahead_size+1)⌉ bits for length, and 8 bits for
next_symbol (ASCII). This is a simplification but good for comparison.

In [27]:
import math

def bit_cost(triples, window_size, lookahead_size, alphabet_bits=8):
    """Compute total bit size of LZ77 triples given field widths."""
    # Bits to encode offset and length (include +1 for zero values)
    b_off = math.ceil(math.log2(window_size + 1))     # offset field width
    b_len = math.ceil(math.log2(lookahead_size + 1))  # length field width
    total_bits = 0
    for off, ln, c in triples:
        # offset + length + optional next symbol (if not end-of-stream '')
        total_bits += b_off + b_len + (alphabet_bits if c != '' else 0)
    return total_bits

def compress_and_report(s, window_size=128, lookahead_size=32):
    """Run LZ77, estimate bit cost, and return a small report."""
    enc = lz77_encode(s, window_size, lookahead_size)            # encode to triples
    bits = bit_cost(enc, window_size, lookahead_size, 8)         # estimated bits
    raw_bits = len(s) * 8                                        # 8 bits per char
    ratio = bits / raw_bits if raw_bits > 0 else float('inf')    # compression ratio
    return {
        'triples': len(enc),             # number of emitted triples
        'compressed_bits': bits,         # estimated compressed size (bits)
        'raw_bits': raw_bits,            # original size (bits)
        'ratio': ratio                   # <1 is good compression, >1 is expansion
    }

# Example usage
report = compress_and_report('abracadabra abracadabra', 64, 16)
report  # quick look at metrics for the teacher

{'triples': 7,
 'compressed_bits': 132,
 'raw_bits': 184,
 'ratio': 0.717391304347826}

### Step 5 — Compare random vs repetitive strings
Run the encoder on both sources and compare compressed size and ratio. LZ77 should compress
the repetitive string substantially better than the random one.

In [28]:
cfg = dict(window_size=128, lookahead_size=32)  # LZ77 config (search/lookahead sizes)
rep_metrics = compress_and_report(S_rep, **cfg)  # metrics for the repetitive string
rnd_metrics = compress_and_report(S_rand, **cfg) # metrics for the random string

# Expectation: S_rep compresses better (lower ratio) than S_rand
print('Repetitive:', rep_metrics)  # shows #triples, compressed_bits, raw_bits, ratio
print('Random: ', rnd_metrics)     # compare ratios: <1 = compression, >1 = expansion

Repetitive: {'triples': 67, 'compressed_bits': 1466, 'raw_bits': 16000, 'ratio': 0.091625}
Random:  {'triples': 942, 'compressed_bits': 20716, 'raw_bits': 16000, 'ratio': 1.29475}


### Step 6 — Parameter study (window/lookahead)
Vary window_size and lookahead_size to see their effect on compression. Larger windows help
find longer back‑references but increase header bit cost per triple.

In [29]:
def sweep_params(s, windows=(32,64,128,256,512), looks=(8,16,32,64)):
    """Grid-search LZ77 params; return rows of (win, look, triples, bits, raw, ratio)."""
    rows = []
    for w in windows:                 # iterate window sizes
        for L in looks:               # iterate lookahead sizes
            m = compress_and_report(s, w, L)  # run encoder + metrics
            rows.append((w, L, m['triples'], m['compressed_bits'], m['raw_bits'], m['ratio']))
    return rows

# Evaluate across parameter grid for both strings
rows_rep = sweep_params(S_rep)
rows_rnd = sweep_params(S_rand)

# Pick top-5 configurations with the lowest ratio (best compression)
rows_rep_sorted = sorted(rows_rep, key=lambda r: r[-1])[:5]
rows_rnd_sorted = sorted(rows_rnd, key=lambda r: r[-1])[:5]

print('Best (repetitive):')
for r in rows_rep_sorted:
    print({'window': r[0], 'look': r[1], 'ratio': round(r[-1], 4)})

print('\nBest (random):')
for r in rows_rnd_sorted:
    print({'window': r[0], 'look': r[1], 'ratio': round(r[-1], 4)})

Best (repetitive):
{'window': 32, 'look': 64, 'ratio': 0.0481}
{'window': 64, 'look': 64, 'ratio': 0.0504}
{'window': 128, 'look': 64, 'ratio': 0.0527}
{'window': 256, 'look': 64, 'ratio': 0.055}
{'window': 512, 'look': 64, 'ratio': 0.0573}

Best (random):
{'window': 512, 'look': 8, 'ratio': 1.1151}
{'window': 256, 'look': 8, 'ratio': 1.1603}
{'window': 512, 'look': 16, 'ratio': 1.1658}
{'window': 128, 'look': 8, 'ratio': 1.177}
{'window': 64, 'look': 8, 'ratio': 1.1958}
